In [1]:
CSS = """
/* source: https://github.com/wch/shiny-wordle/blob/main/app-final.R */

#container{
    border-style: groove;
    border-color:#888;
    display: inline-block;
    margin: 1px;
    width: 50px;
    height: 50px;
    text-align: center;
    vertical-align: middle;
    border-radius: 3px;
    line-height: 50px;
    font-size: 32px;
    font-weight: bold;
    vertical-align: middle;
    user-select: none;
    color: white;
    font-family: 'Clear Sans', 'Helvetica Neue', Arial, sans-serif;
    background-color: white;
}

#guess-correct{
    margin: 1px;
    display: inline-block;
    width: 50px;
    height: 50px;
    text-align: center;
    vertical-align: middle;
    border-radius: 3px;
    line-height: 50px;
    font-size: 32px;
    font-weight: bold;
    vertical-align: middle;
    user-select: none;
    color: white;
    font-family: 'Clear Sans', 'Helvetica Neue', Arial, sans-serif;
    background-color: #6a5;
}

#guess-in-word{
    margin: 1px;
    display: inline-block;
    width: 50px;
    height: 50px;
    text-align: center;
    vertical-align: middle;
    border-radius: 3px;
    line-height: 50px;
    font-size: 32px;
    font-weight: bold;
    vertical-align: middle;
    user-select: none;
    color: white;
    font-family: 'Clear Sans', 'Helvetica Neue', Arial, sans-serif;
    background-color: #db5;
}

#guess-not-word{
    margin: 1px;
    display: inline-block;
    width: 50px;
    height: 50px;
    text-align: center;
    vertical-align: middle;
    border-radius: 3px;
    line-height: 50px;
    font-size: 32px;
    font-weight: bold;
    vertical-align: middle;
    user-select: none;
    color: white;
    font-family: 'Clear Sans', 'Helvetica Neue', Arial, sans-serif;
    background-color: #888;
}


.keyboard {
    height: 240px;
    user-select: none;
}
.keyboard .keyboard-row {
    margin: 3px;
}
.keyboard .keyboard-row .key {
    display: inline-block;
    padding: 0;
    margin: 3px;
    width: 30px;
    height: 50px;
    text-align: center;
    vertical-align: middle;
    border-radius: 3px;
    line-height: 50px;
    font-size: 18px;
    font-weight: bold;
    vertical-align: middle;
    color: black;
    font-family: 'Clear Sans', 'Helvetica Neue', Arial, sans-serif;
    background-color: #ddd;
    touch-action: none;
}
.keyboard .keyboard-row .key:focus {
    outline: none;
}
.keyboard .keyboard-row .key.wide-key {
    font-size: 15px;
    width: 50px;
}
.keyboard .keyboard-row .key.correct {
    background-color: #6a5;
    color: white;
}
.keyboard .keyboard-row .key.in-word {
    background-color: #db5;
    color: white;
}
.keyboard .keyboard-row .key.not-in-word {
    background-color: #888;
    color: white;
}
"""

In [2]:
import panel as pn
import random
from words import words_common, words_all
pn.extension(raw_css=[CSS])

all_words = words_all
word = random.choice(words_common)

inp = pn.widgets.TextInput(placeholder='Enter your guess')
button = pn.widgets.Button(name='Go!', disabled=True)
html = pn.pane.HTML(sizing_mode='stretch_width')

#grid
grid = """
<center>
"""
container_row = """
<div id="container"> _ </div>
<div id="container"> _ </div>
<div id="container"> _ </div>
<div id="container"> _ </div>
<div id="container"> _ </div>
<br>
"""
grid += 6*container_row

# keyboard
keys = [
    ("Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"),
    ("A", "S", "D", "F", "G", "H", "J", "K", "L"),
    ("Enter", "Z", "X", "C", "V", "B", "N", "M", "Back")
]
keyboard = '<div id="keyboard" class="keyboard">'
for row in keys:
    keyboard += '<div class="keyboard-row">'
    for k in row: 
        if k in ["Enter", "Back"]:
            keyboard += f'<div id="key-{k}" class="key wide-key">{k}</div>'
        else:
            keyboard += f'<div id="key-{k}" class="key">{k}</div>'
    keyboard += '</div>'
keyboard += '</div>'

html.object=grid + keyboard + '</center>'

def disable_button(event):
    button.disabled = event.new not in all_words

inp.param.watch(disable_button, 'value_input')

display_all = """
<center>
"""
guesses = []
letter_correct = []
letter_in_word = []
letter_not_word = []
def update(event):
    global display_all
    global keyboard
    guess = inp.value
    inp.value = ''
    display = ''
    for g, w in zip(guess, word):
        if g == w:
            display += f"""
            <div id="guess-correct"> {g} </div>
            """
            letter_correct.append(g)
        elif g in word:
            display += f"""
            <div id="guess-in-word"> {g} </div>
            """
            letter_in_word.append(g)
        else:
            display += f"""
            <div id="guess-not-word"> {g} </div>
            """
            letter_not_word.append(g)
    display += """<br>"""
    display_all += display
    guesses.append(guess)
    
    display_grid = display_all + (6-len(guesses)) * container_row
    
    keyboard = '<div id="keyboard" class="keyboard">'
    for row in keys:
        keyboard += '<div class="keyboard-row">'
        for k in row: 
            if k in ["Enter", "Back"]:
                keyboard += f'<div id="key-{k}" class="key wide-key">{k}</div>'
            elif k.lower() in letter_correct: 
                keyboard += f'<div id="key-{k}" class="key correct">{k}</div>'
            elif k.lower() in letter_in_word: 
                keyboard += f'<div id="key-{k}" class="key in-word">{k}</div>'
            elif k.lower() in letter_not_word: 
                keyboard += f'<div id="key-{k}" class="key not-in-word">{k}</div>'
            else:
                keyboard += f'<div id="key-{k}" class="key">{k}</div>'
        keyboard += '</div>'
    keyboard += '</div>'
    
    html.object = display_grid + keyboard + '</center>'

button.on_click(update)

dash = pn.Row(
    pn.Column(inp, button),
    html,   
)

In [4]:
dash

Row
    [0] Column
        [0] TextInput(placeholder='Enter your guess', value_input='panel')
        [1] Button(clicks=1, name='Go!')
    [1] HTML(str, sizing_mode='stretch_width')